### Code to obtain table of Best Performing Districts

This is done with a function that computes a table based on the districts that have best improvement for every combination of Shipment Level and Day.
improvement is with respect to Recalculation Solver to DP Sequence (absolut difference and percentage)

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import scipy
import seaborn as sns

In [3]:
#Set options in pandas to be able to see a complete dataframe without truncating values for display purpose
pd.set_option('display.max_rows', None)

In [4]:
#Define your default paths for the complete converged KPI's CSV file

path_converged = r'C:\Users\julli\OneDrive\Escritorio\DP_DATA_TEST\KPI Converged Values'

path_equal_kpi = os.path.join(path_converged, 'all_equal_KPI_converged_data.csv')
path_unequal_kpi = os.path.join(path_converged, 'all_unequal_KPI_converged_data.csv')


df_equal_kpi_complete = pd.read_csv(path_equal_kpi)
df_unequal_kpi_complete = pd.read_csv(path_unequal_kpi)

In [5]:
region_color_dict = {'Warmsen': 'orange',
                     'Uerze': 'green',
                     'Hannover 92': 'blue'}

In [12]:
def create_colored_district_table(df_district_table, dict_colors):

    # Merge 'Color' and 'Value' columns into one single column
    df_district_table['District'] =  list(zip(df_district_table.Region, df_district_table.District))

    # Perform pivot operation
    df_equal_pics = df_district_table.pivot(index= 'Shipment_level', columns = 'Day', values = 'District')

    # Split into two dataframes: a colors dataframe and a numerical values dataframe
    color_df = df_equal_pics.applymap(lambda x: x[0])
    value_df = df_equal_pics.applymap(lambda x: x[1])


    # Transform dataframe with colors into formatting commands
    color_df = color_df.applymap(lambda x: f'background-color: {dict_colors[x]}')


    # Apply color styling to values dataframe
    df_colored_equal = value_df.style.apply(lambda x: color_df, axis=None)

    return df_colored_equal

In [23]:
def get_district_table(kpi_converged_table, region_colors):

    kpi_converged_table['Distance Improvement Recalc vs DP'] = kpi_converged_table['Avg Distance DP'] - kpi_converged_table['Avg Distance Recalc']

    kpi_converged_table_reduced = pd.DataFrame().assign(Region = kpi_converged_table['Region'], 
                                                        District = kpi_converged_table['District'], 
                                                        Day = kpi_converged_table['Day'], 
                                                        Shipment_level = kpi_converged_table['Shipment Level'],
                                                        Improvement_Value = kpi_converged_table['Distance Improvement Recalc vs DP'],
                                                        Improvement_Pctg = kpi_converged_table['Cost Improvement % Recalc vs Solver'])
    

    
    df_max_value_districts = pd.DataFrame()
    df_max_pctg_districts = pd.DataFrame()
    
    day_list = kpi_converged_table_reduced['Day'].unique().tolist()
    for day in day_list: 
        level_list = kpi_converged_table_reduced['Shipment_level'].unique().tolist()
        for level in level_list: 
            df_A = kpi_converged_table_reduced[(kpi_converged_table_reduced['Day'] == day) & (kpi_converged_table_reduced['Shipment_level'] == level)]
            max_row_value = df_A.loc[df_A['Improvement_Value'].idxmax()].to_frame().transpose()
            max_row_pctg = df_A.loc[df_A['Improvement_Pctg'].idxmax()].to_frame().transpose()
            df_max_value_districts = pd.concat([df_max_value_districts, max_row_value], ignore_index=True)
            df_max_pctg_districts = pd.concat([df_max_pctg_districts, max_row_pctg], ignore_index=True)

    df_max_value_districts = df_max_value_districts.convert_dtypes()
    df_max_pctg_districts = df_max_pctg_districts.convert_dtypes()

    df_max_value_districts_color = create_colored_district_table(df_district_table = df_max_value_districts, dict_colors= region_colors)
    df_max_pctg_districts_color = create_colored_district_table(df_district_table = df_max_pctg_districts, dict_colors= region_colors)

    Max_District_Value_Instance = df_max_value_districts.loc[df_max_value_districts['Improvement_Value'].idxmax()].to_frame().transpose()
    Max_District_Pctg_Instance = df_max_pctg_districts.loc[df_max_pctg_districts['Improvement_Pctg'].idxmax()].to_frame().transpose()

    return df_max_value_districts_color, df_max_pctg_districts_color, Max_District_Value_Instance, Max_District_Pctg_Instance

In [27]:
#Get Results of Table and Specific Best Improvement Instance for Equal Instances
equal_district_values, equal_district_pctg, equal_best_district_value, equal_best_district_pctg = get_district_table(kpi_converged_table = df_equal_kpi_complete, region_colors = region_color_dict)

In [28]:
print('Equal: District and Instance with Highest Improvement by Value:')
equal_best_district_value

Equal: District and Instance with Highest Improvement by Value:


,Region,District,Day,Shipment_level,Improvement_Value,Improvement_Pctg
10,Warmsen,"(Warmsen, 31606-11)",Friday,90,2139.71,9.82


In [29]:
equal_district_values

Day,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday
Shipment_level,,,,,,
5,30459-24,30457-12,31606-11,30459-24,31606-11,30459-24
10,31606-11,31606-11,31606-11,31606-11,31606-11,31606-11
20,31606-11,31606-11,31606-11,31606-11,31606-11,31606-11
30,31606-11,31606-11,31606-11,31606-11,31606-11,31606-11
40,31606-11,31606-11,31606-11,31606-11,31606-11,31606-11
50,31606-11,31606-11,31606-11,31606-11,31606-11,31606-11
60,31606-11,31606-11,31606-11,31606-11,31606-11,31606-11
70,31606-11,31606-11,31606-11,31606-11,31606-11,31606-11
80,31606-11,31606-11,31606-11,31606-11,31606-11,31606-11


In [30]:
print('Equal: District and Instance with Highest Improvement by Percentage:')
equal_best_district_pctg

Equal: District and Instance with Highest Improvement by Percentage:


,Region,District,Day,Shipment_level,Improvement_Value,Improvement_Pctg
15,Warmsen,"(Warmsen, 31600-03)",Monday,40,729.89,29.13


In [31]:
equal_district_pctg

Day,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday
Shipment_level,,,,,,
5,31600-03,31311-04,31600-03,31600-03,31606-11,31600-03
10,31604-10,31311-04,31604-10,31604-10,31600-03,31604-10
20,31604-10,31600-03,31604-10,31604-10,31604-10,31604-10
30,31604-10,31600-03,31604-10,31604-10,31604-10,31604-10
40,31604-10,31600-03,31604-10,31604-10,31604-10,31604-10
50,31604-10,31600-03,31604-10,31604-10,31604-10,31604-10
60,31604-10,31600-03,31604-10,31604-10,31604-10,31604-10
70,31603-05,31604-10,31604-10,31604-10,31603-05,31604-10
80,31604-10,31604-10,31604-10,31604-10,31604-10,31604-10


In [32]:
#Get Results of Table and Specific Best Improvement Instance for Unequal Instances

unequal_district_values, unequal_district_pctg, unequal_best_district_value, unequal_best_district_pctg = get_district_table(kpi_converged_table = df_unequal_kpi_complete, region_colors = region_color_dict)

In [33]:
print('Unequal: District and Instance with Highest Improvement by Value:')
unequal_best_district_value

Unequal: District and Instance with Highest Improvement by Value:


,Region,District,Day,Shipment_level,Improvement_Value,Improvement_Pctg
0,Warmsen,"(Warmsen, 31606-11)",Friday,Sce_0,2103.75,9.53


In [35]:
unequal_district_values

Day,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday
Shipment_level,,,,,,
Sce_0,31606-11,31606-11,31606-11,31606-11,31606-11,31606-11
Sce_1,31606-11,31606-11,31600-03,31606-11,31606-11,31606-11
Sce_2,31606-11,31606-11,31606-11,31606-11,31606-11,31606-11


In [36]:
print('Equal: District and Instance with Highest Improvement by Percentage:')
unequal_best_district_pctg

Equal: District and Instance with Highest Improvement by Percentage:


,Region,District,Day,Shipment_level,Improvement_Value,Improvement_Pctg
4,Warmsen,"(Warmsen, 31600-03)",Monday,Sce_1,1062.19,24.29


In [37]:
unequal_district_pctg

Day,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday
Shipment_level,,,,,,
Sce_0,31604-10,31604-10,31604-10,31604-10,31604-10,31604-10
Sce_1,31604-10,31600-03,31604-10,31604-10,31604-10,31604-10
Sce_2,31604-10,31604-10,31603-05,31604-10,31604-10,31604-10
